# Homework I

Gonçalo Bárias (ist1103124) & Raquel Braunschweig (ist1102624)

## I. Pen and Paper [11v]

**Consider the partially learnt decision tree from the dataset 𝐷. 𝐷 is described by four input variables –
one numeric with values in [0,1] and 3 categorical – and a target variable with three classes.**

### 1) [5v] 

**Complete the given decision tree using Information gain with Shannon entropy (log2). Consider that:**

**i) A minimum of 4 observations is required to split an internal node;**

**ii) Decisions by ascending alphabetic order should be placed in case of ties.**

### 2) [2.5v] 

**Draw the training confusion matrix for the learnt decision tree.**

<table>
  <tr>
    <td colspan="2" rowspan="2" style="border-top: none; border-left: none;"></td>
    <th colspan="2">True</th>
    <td rowspan="2" style="border-top: none; border-right: none;"></td>
  </tr>
  <tr>
    <th>Positive</th>
    <th>Negative</th>
  </tr>
  <tr>
    <th rowspan="2">Predicted</th>
    <th>Positive</th>
    <td>A</td>
    <td>B</td>
    <td>C</td>
  </tr>
  <tr>
    <th>Negative</th>
    <td>D</td>
    <td>E</td>
    <td>F</td>
  </tr>
  <tr>
    <th colspan="2" style="border-left: none; border-bottom: none;"></th>
    <td>G</td>
    <td>H</td>
    <td>I</td>
  </tr>
</table>

### 3) [1.5v] 

**Identify which class has the lowest training F1 score.**

### 4) [1v] 

**Considering y2 to be ordinal, assess if y1 and y2 are correlated using the Spearman coefficient.**

### 5) [1v] 

**Draw the class-conditional relative histograms of y1 using 5 equally spaced bins in [0,1]. Find the root split using the discriminant rules from these empirical distributions.**

## II. Programming [9v]

**Consider the `column_diagnosis.arff` data available at the homework tab, comprising 6 biomechanical features to classify 310 orthopaedic patients into 3 classes (`normal`, `disk hernia`, `spondilolysthesis`).**

### 1) [1.5v]

**Apply `f_classif` from `sklearn` to assess the discriminative power of the input variables.
Identify the input variable with the highest and lowest discriminative power.
Plot the class-conditional probability density functions of these two input variables.**

### 2) [4v]

**Using a stratified 70-30 training-testing split with a fixed seed (`random_state=0`), assess in a
single plot both the training and testing accuracies of a decision tree with depth limits in
`{1,2,3,4,5,6,8,10}` and the remaining parameters as default.**

***[Optional]* Note that split thresholding of numeric variables in decision trees is non-deterministic
in sklearn, hence you may opt to average the results using 10 runs per parameterization.**

### 3) [1.5v]

**Comment on the results, including the generalization capacity across settings.**

### 4) [2v]

**To deploy the predictor, a healthcare team opted to learn a single decision tree
(`random_state=0`) using *all* available data as training data, and further ensuring that each leaf has
a minimum of 20 individuals in order to avoid overfitting risks.**

- **i. Plot the decision tree.**

- **ii. Characterize a hernia condition by identifying the hernia-conditional associations.**